In [ ]:

# 📊 FC25 Sentiment Prediction Notebook

# ✅ STEP 1: Upload Required Files
from google.colab import files
uploaded = files.upload()

# Required:
# - model.pkl
# - vectorizer.pkl
# - raw_comments.csv

# ✅ STEP 2: Load Model and Vectorizer
import pickle
import pandas as pd

if "model.pkl" not in uploaded or "vectorizer.pkl" not in uploaded or "raw_comments.csv" not in uploaded:
    raise Exception("Please upload model.pkl, vectorizer.pkl, and raw_comments.csv to proceed.")

with open("model.pkl", "rb") as f:
    model = pickle.load(f)

with open("vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

# ✅ STEP 3: Load and Clean Raw Comments
df = pd.read_csv("raw_comments.csv")
if df.columns[0] != "Comment":
    df.columns = ["Comment"]

df = df.drop_duplicates()
df = df[df["Comment"].str.strip() != ""]

# ✅ STEP 4: Predict Sentiment with Confidence Threshold
X_new = vectorizer.transform(df["Comment"])
probs = model.predict_proba(X_new)
preds = model.predict(X_new)
max_probs = probs.max(axis=1)

confidence_threshold = 0.60
final_labels = [label if prob >= confidence_threshold else "Uncertain"
                for label, prob in zip(preds, max_probs)]

df["Sentiment"] = final_labels

# ✅ STEP 5: Export Labeled Comments
output_path = "labeled_comments.csv"
df.to_csv(output_path, index=False)
files.download(output_path)
